In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [20]:
url_inicial = 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/archive/2'
url_root = 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/archive/2'
r=requests.get(url_inicial)

In [21]:
soup = BeautifulSoup(r.text, 'html.parser')

In [22]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/234',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/233',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/232',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/231',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/230',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/229',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/228',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/227',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/226',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/view/225',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue

In [23]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [24]:
vol2

['https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5876',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5877',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5859',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5840',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5841',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5842',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5823',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5824',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5825',
 'https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/article/view/5806',
 'https://revistas.juridicas.unam.mx/index.php/cue

In [25]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [26]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://revistas.juridicas.unam.mx/index.php/cuestiones-constitucionales/issue/archive/2


In [27]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

45

In [28]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [29]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Cuestiones Constitucionales. Revista Mexicana de Derecho Constitucional'
    a='V. Ciencias Sociales'
    tem='Derecho Constitucional, Derechos Humanos, Derecho Procesal Constitucional, Justicia Constitucional, Constitucionalismo Contemporaneo'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=resu#.replace("Resumen","ABSTRACT","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [30]:
list_scraper=list_scraper[0:45]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [31]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Preliminares,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
1,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Los autores,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
2,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Preliminares,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
3,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Preliminares,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
4,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Normas para la presentación de colaboraciones,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
5,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Guidelines for submitting manuscripts,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
6,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Preliminares,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
7,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Normas para la presentación de colaboraciones,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
8,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Guidelines for submitting manuscripts,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...
9,Cuestiones Constitucionales. Revista Mexicana ...,V. Ciencias Sociales,"Derecho Constitucional, Derechos Humanos, Dere...",Preliminares,,None,https://revistas.juridicas.unam.mx/index.php/c...,https://revistas.juridicas.unam.mx/index.php/c...


In [32]:
df_catalogo.to_csv('Revista63.02.csv', index=False)